In [55]:
from autogen import UserProxyAgent, AssistantAgent, GroupChat, GroupChatManager,ConversableAgent

In [ ]:
llm_config = {
    "config_list": [
        {
            "model": "mistralai/mistral-7b-instruct",
            "api_key": "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",
            "base_url": "https://openrouter.ai/api/v1",
            "price": [0.0, 0.0]  
        }
    ],
    "temperature": 0.7,
    "cache_seed": 42
}

In [ ]:
user_agent = ConversableAgent(
    name="User",
    system_message="You are a user who wants to plan a trip. You will describe your travel preferences, destination, and budget when the conversation starts.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1
)


itinerary_agent = AssistantAgent(
    name="ItineraryAgent",
    system_message=(
        "You are a travel itinerary planner. Based on the user's destination (already provided), create a detailed 5-day plan. "
        "Distribute time logically between destinations and include major travel legs, local exploration, and some downtime."
    ),
    llm_config=llm_config
)


place_agent = AssistantAgent(
    name="PlaceAgent",
    system_message=(
        "You recommend top attractions and local experiences for each place in the itinerary. Include scenic spots, treks, cultural sites, or hidden gems with a 1-line reason each."
    ),
    llm_config=llm_config
)


hotel_agent = AssistantAgent(
    name="HotelAgent",
    system_message=(
        "You suggest 1–2 reliable budget hotel options (₹1000–₹2000 per night) for each stop in the itinerary. Consider location, safety, cleanliness, and traveler reviews."
    ),
    llm_config=llm_config
)


transport_agent = AssistantAgent(
    name="TransportAgent",
    system_message=(
        "You plan transportation between the listed places. Include mode (bus, cab, etc.), estimated time and cost. Consider terrain and efficiency."
    ),
    llm_config=llm_config
)


budget_agent = AssistantAgent(
    name="BudgetAgent",
    system_message=(
        "Estimate the total cost of the trip using hotel, transport, food, and activities. Keep the total around ₹30,000. Provide a clear breakdown and flag any overages."
    ),
    llm_config=llm_config
)

summary_agent = AssistantAgent(
    name="SummaryAgent",
    system_message=(
        "You are a travel planner. Summarize the outputs from all other agents (itinerary, places, hotels, transport, and budget) "
        "into a single, well-structured 5-day travel plan for the user. Format it clearly with headings like 'Day-wise Plan', 'Accommodation', 'Transport', and 'Estimated Budget'."
    ),
    llm_config=llm_config
)

In [69]:
groupchat = GroupChat(
    agents=[
        user_agent,
        itinerary_agent,
        place_agent,
        hotel_agent,
        transport_agent,
        budget_agent,
        summary_agent
    ],
    messages=[],
    max_round=7,  
    allow_repeat_speaker=False 
)

manager = GroupChatManager(
    groupchat=groupchat,
    name="TravelPlanner",
    llm_config=llm_config
)

In [71]:
user_agent.initiate_chat(
    manager,
    message="I want to plan a 5-day trip to London. My budget is ₹2,00,000. I'm interested in history, museums, walking tours, and local food. Please help with the full plan."
)

User (to TravelPlanner):

I want to plan a 5-day trip to London. My budget is ₹2,00,000. I'm interested in history, museums, walking tours, and local food. Please help with the full plan.

--------------------------------------------------------------------------------

Next speaker: ItineraryAgent

ItineraryAgent (to TravelPlanner):

 Title: 5-Day London Itinerary for History, Museums, Walking Tours, and Local Food Exploration (Budget: ₹2,00,000)

Day 1: Arrival in London

- Arrival at Heathrow Airport (LHR) (Assume flight cost is included in the budget)
- Transfer to Hotel (≈ 1 hr 30 min by Heathrow Express, ₹4,500 for a return ticket)
   - Recommended Hotel: The Rubens at the Palace (4-star, £250 per night)

12:00 PM: Lunch at Borough Market (Traditional British food)
1:30 PM: Visit The London Eye (₹3,500 per ride)
   - Offers stunning views of the city and landmarks

4:00 PM: Walk along the South Bank to the London Dungeon (₹2,500 per ticket)
   - Interactive, historical, and enter

ChatResult(chat_id=None, chat_history=[{'content': "I want to plan a 5-day trip to London. My budget is ₹2,00,000. I'm interested in history, museums, walking tours, and local food. Please help with the full plan.", 'role': 'assistant', 'name': 'User'}, {'content': " Title: 5-Day London Itinerary for History, Museums, Walking Tours, and Local Food Exploration (Budget: ₹2,00,000)\n\nDay 1: Arrival in London\n\n- Arrival at Heathrow Airport (LHR) (Assume flight cost is included in the budget)\n- Transfer to Hotel (≈ 1 hr 30 min by Heathrow Express, ₹4,500 for a return ticket)\n   - Recommended Hotel: The Rubens at the Palace (4-star, £250 per night)\n\n12:00 PM: Lunch at Borough Market (Traditional British food)\n1:30 PM: Visit The London Eye (₹3,500 per ride)\n   - Offers stunning views of the city and landmarks\n\n4:00 PM: Walk along the South Bank to the London Dungeon (₹2,500 per ticket)\n   - Interactive, historical, and entertaining experience\n\n7:00 PM: Dinner at Gordon Ramsay's 

In [72]:
for msg in groupchat.messages[::-1]:  # reverse for latest message first
    if msg['name'] == "SummaryAgent":
        print("\n\nFINAL TRAVEL PLAN:\n")
        print(msg["content"])
        break



FINAL TRAVEL PLAN:

  Here's a revised itinerary that stays within your budget of ₹2,00,000. I've made adjustments to the hotel, food, and activities to help you save costs.

Title: 5-Day London Itinerary for History, Museums, Walking Tours, and Local Food Exploration (Budget: ₹2,00,000)

Day 1: Arrival in London
- Arrival at Heathrow Airport (LHR) (Assume flight cost is included in the budget)
- Transfer to Hotel (≈ 1 hr 30 min by Heathrow Express, ₹4,500 for a return ticket)
   - Recommended Hotel: Park Plaza Victoria (4-star, £150 per night)

12:00 PM: Lunch at Borough Market (Traditional British food)
1:30 PM: Visit The London Eye (₹3,500 per ride)
   - Offers stunning views of the city and landmarks

4:00 PM: Walk along the South Bank to the London Dungeon (₹2,500 per ticket)
   - Interactive, historical, and entertaining experience

7:00 PM: Dinner at Wahaca (₹2,500 for two)
   - Authentic Mexican cuisine with a British twist

Day 2: Historical Sites and Museums
8:00 AM: Breakf

In [74]:
# print("\n\nFULL CHAT TRANSCRIPT:\n")
# for msg in groupchat.messages:
#     speaker = msg.get("name", "Unknown")
#     role = msg.get("role", "agent")
#     content = msg.get("content", "")
#     print(f"\n👤 {speaker} ({role}):\n{content}\n{'-'*60}")